在项目代码文件夹添加一个`.env`文件存放API密钥，配置gitignore防止敏感信息泄露，同时方便项目隔离，使用python的dotenv模块读取，使文件中的环境变量成为进程的环境变量

OPENAI_API_KEY="xxxxxxxx"

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

openai_api_key=os.getenv('OPENAI_API_KEY')

In [1]:
!pip install openai langchain langchain_openai

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 42.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 597.4/597.4 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.0/782.0 kB 38.9 MB/s eta 0:00:00


此处使用智谱GLM-4模型，文档见：https://maas.aminer.cn/dev/api/normal-model/glm-4
模型的每次调用是无状态的，不会记得之前说了什么，能够形成完整对话是因为还传输了过往的对话，过往的对话会存在message中（一个字典组成的列表），key为role（谁说话，取值role、system给整个对话奠定基调指定扮演角色完成任务，只能出现在第一个只能出现一次、assistant）和content（说了什么）

In [11]:
from openai import OpenAI
client=OpenAI(
    base_url="https://open.bigmodel.cn/api/paas/v4/"
)

completion=client.chat.completions.create(
    model='glm-4',
    messages=[
        {'role':'system','content':"你被用于抑制用户的购买欲望。当用户说想要买什么东西的时候，你需要提供理由让用户不要买"},
        {'role':'user','content':"我正准备购买一台单反，你需要帮助我思考一下是否真的需要这个商品"}
    ],
    # 限制输出token数量，简短/冗长的选择
    max_tokens=300,
    # 随机性，更低的值生成更不变的输出
    temperature=0.7
)
print(completion.choices[0].message.content)

当然，让我们一起来考虑一下你购买单反相机的决定。以下是一些可能让你重新思考的理由：

1. **成本问题**：单反相机及其配件价格不菲。你是否有足够的预算来承担这笔开销？此外，高质量的镜头和其他配件也会增加成本。

2. **使用频率**：你真的会频繁使用这台单反相机吗？如果你只是偶尔拍照，可能不需要这么专业的设备。

3. **技术知识**：单反相机通常需要一定的摄影知识和技能才能充分发挥其功能。如果你没有时间或兴趣去学习这些技能，可能会觉得使用起来有难度。

4. **便携性**：单反相机通常比普通相机或手机重且体积大，这可能使得携带不便，影响你拍照的意愿。

5. **维护和保养**：专业相机需要定期的清洁和保养，这需要额外的时间和金钱。

6. **手机相机的进步**：现代智能手机的相机功能已经非常强大，对于日常拍照来说，它们可能已经足够好。

7. **现有设备**：你目前是否已经有了可以满足你摄影需求的设备？如果没有，单反是否是唯一的选择？

8. **长期兴趣**：购买单反相机是一笔长期投资，你对摄影的兴趣是否足够持久？

9. **替代方案**：是否有其他更经济、更实用的替代品，比如高级紧凑型相机或微单？

10. **环境因素**：考虑到环保，减少消费和浪费是


这样就能调用api来使用ai了，调整参数来适应使用的场景
完整的调用包含：
1. prompt template，prompt是输入给模型的内容，设置template将prompt中不变的地方固定住，变的地方留空
2. model 
3. output parser 进一步解析模型的输出，尤其是当模型的输出和我的需求格式不一致时，过程可能再次调用模型
这个pipline可用封装好的langchain进行使用

In [10]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# 商品部分留空，调用的时候填充
prompt_template=ChatPromptTemplate.from_messages(
    [
        ('system',"你被用于抑制用户的购买欲望。当用户说想要买什么东西的时候，你需要提供理由让用户不要买"),
        ('human',"我正准备购买一个{product}，你需要帮助我思考一下是否真的需要这个商品")
    ]
)
prompt_template.format(product='显示器')

model=ChatOpenAI(
    model='glm-4',
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/",
    max_tokens=300,
    temperature=0.7)

# 工程中想要获得稳定一致的输出，一个步骤最好只做一件事
def output_parser(output:str):
    parser_model=ChatOpenAI(
    model='glm-4',
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/",
    temperature=0.6)
    message="你需要将传入的文本改写，表达尽可能自然。这是你要改写的文本'{text}'"
    return parser_model.invoke(message.format(text=output))

chain=prompt_template|model|output_parser
answer=chain.invoke(input={'product':'耳机'})
print(answer.content)

'content='当然，我们可以一起探讨你打算购买新耳机的决定。以下是几个值得思考的要点，或许能帮你更好地审视你的购物欲望：

1. **现有耳机的使用情况**：你已经有了一副耳机，它们还能用吗？如果它们还能正常使用，或许你并不需要换新。
   
2. **财务规划**：审视一下你的财务状况。是不是有更重要的开销需要优先考虑？也许这笔钱可以用于储蓄或投资。

3. **需求分析**：你买新耳机是出于实际需求，还是仅仅因为追求潮流？如果现有耳机已经满足你的需求，那么升级可能并不必要。

4. **环境保护**：电子产品的生产会对环境产生影响。如果你并非真的需要新耳机，那么避免购买可以减少电子垃圾。

5. **产品耐用性**：虽然新产品很有吸引力，但考虑一下你现有耳机的使用寿命。如果它们还能用很长时间，那么或许可以等到更合适的时机再换。

6. **替代方案**：是否有其他更经济的方法来解决你想要解决的问题？例如，用耳塞清洁工具来改善现有耳机的音质，或者更换耳塞来提升舒适度。

7. **未来开销**：如果你预知自己在不久的将来会有大额支出，那么现在节省资金可能是明智之举。

8. **冲动消费**：最后，考虑这次购买是否属于一时冲动。如果是，那么等待一段时间再决定是否购买，可能会帮助你更清楚地判断自己是否真的需要这副耳机。

通过这些思考，你可以更好地...
